In [1]:
import numpy as np

In [44]:
class Monkey:
    def __init__(self, lines) -> None:
        self.number = int(lines[0][-2])

        start_items_string = lines[1].replace(':', '').replace(',', '')
        self.items = [int(word) for word in start_items_string.split(' ') if word.isdigit()]

        self.operation = lines[2].split(' ')[-2]
        self.operand = lines[2].split(' ')[-1]

        self.test_condition = int(lines[3].split(' ')[-1])
        self.true_monkey = int(lines[4].split(' ')[-1])
        self.false_monkey = int(lines[5].split(' ')[-1])

        self.inspections = 0


    def inspect_items(self):
        for i, item in enumerate(self.items):
            self.inspections += 1
            if self.operation == '+':
                self.items[i] += int(self.operand)
            if self.operation == '*':
                if self.operand.isdigit():
                    self.items[i] *= int(self.operand)
                else:
                    self.items[i] *= item
        
    def reduce_worries(self):
        for i, item in enumerate(self.items):
            self.items[i] = item // 3


    def throw_items(self):
        planned_throws = []
        for item in self.items:
            if item % self.test_condition == 0:
                planned_throws.append((item, self.true_monkey))
            else:
                planned_throws.append((item, self.false_monkey))
        self.items = []
        return planned_throws

    def receive_item(self, item):
        self.items.append(item)

    def print_monkey_items(self):
        print(f"Monkey {self.number}:" + ', '.join([str(item) for item in self.items]))

In [45]:
def solve_part_1(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        lines = [entry.strip() for entry in lines]

    monkeys = []
    while len(lines) > 0:
        monkeys.append(Monkey(lines))
        lines = lines[7:] if len(lines) > 7 else []

    round = 0
    while round < 20:
        round += 1
        for monkey in monkeys:
            monkey.inspect_items()
            monkey.reduce_worries()
            planned_throws = monkey.throw_items()
            for item, target_monkey in planned_throws:
                monkeys[target_monkey].receive_item(item)
    print(f"After round {round}:")
    for monkey in monkeys:
        monkey.print_monkey_items()

    inspections = [monkey.inspections for monkey in monkeys]
    print(np.prod(sorted(inspections, reverse=True)[:2]))

In [46]:
solve_part_1('example.txt')

After round 20:
Monkey 0:10, 12, 14, 26, 34
Monkey 1:245, 93, 53, 199, 115
Monkey 2:
Monkey 3:
10605


# Part 2

In [47]:
class MonkeyItem:
    def __init__(self, divisors, start_value) -> None:
        self.values_per_monkey = []
        self.divisors = divisors
        for divisor in divisors:
            self.values_per_monkey.append(start_value % divisor)
        
    def add_value(self, addon):
        for i, value in enumerate(self.values_per_monkey):
            self.values_per_monkey[i] = (value + addon) % self.divisors[i]
    
    def square_value(self):
        for i, value in enumerate(self.values_per_monkey):
            self.values_per_monkey[i] = (value**2) % self.divisors[i]
    
    def multiply_value(self, multiplier):
        for i, value in enumerate(self.values_per_monkey):
            self.values_per_monkey[i] = (value*multiplier) % self.divisors[i]

In [48]:
class Monkey2:
    def __init__(self, lines) -> None:
        self.number = int(lines[0][-2])

        start_items_string = lines[1].replace(':', '').replace(',', '')
        self.items = [int(word) for word in start_items_string.split(' ') if word.isdigit()]

        self.operation = lines[2].split(' ')[-2]
        self.operand = lines[2].split(' ')[-1]

        self.test_condition = int(lines[3].split(' ')[-1])
        self.true_monkey = int(lines[4].split(' ')[-1])
        self.false_monkey = int(lines[5].split(' ')[-1])

        self.inspections = 0

    def convert_item_values(self, divisors):
        new_items = []
        for item_value in self.items:
            new_items.append(MonkeyItem(divisors, item_value))
        self.items = new_items

    def inspect_items(self):
        for i, item in enumerate(self.items):
            self.inspections += 1
            if self.operation == '+':
                self.items[i].add_value(int(self.operand))
            if self.operation == '*':
                if self.operand.isdigit():
                    self.items[i].multiply_value(int(self.operand))
                else:
                    self.items[i].square_value()


    def throw_items(self):
        planned_throws = []
        for item in self.items:
            if item.values_per_monkey[self.number] == 0:
                planned_throws.append((item, self.true_monkey))
            else:
                planned_throws.append((item, self.false_monkey))
        self.items = []
        return planned_throws

    def receive_item(self, item):
        self.items.append(item)

    def print_monkey_items(self):
        print(f"Monkey {self.number}:" + ', '.join([str(item.values_per_monkey) for item in self.items]))

In [49]:
def solve_part_2(file):
    with open(file, 'r') as f:
        lines = f.readlines()
        lines = [entry.strip() for entry in lines]

    monkeys = []
    while len(lines) > 0:
        monkeys.append(Monkey2(lines))
        lines = lines[7:] if len(lines) > 7 else []
    
    divisors = test_conditions = [monkey.test_condition for monkey in monkeys]
    for monkey in monkeys:
        monkey.convert_item_values(divisors)

    round = 0
    while round < 10000:
        round += 1
        for monkey in monkeys:
            monkey.inspect_items()
            planned_throws = monkey.throw_items()
            for item, target_monkey in planned_throws:
                monkeys[target_monkey].receive_item(item)

    inspections = [monkey.inspections for monkey in monkeys]
    print(np.prod(sorted(inspections, reverse=True)[:2]))

In [50]:
solve_part_2('example.txt')

2713310158
